In [1]:
import torch
import random
from old_seeker import White_seeker, Random_select_seeker, Random_gen_seeker, Black_seeker, Test_seeker, Norm_Test_seeker, Random_gen_gradiant_seeker
from seeker.gradiant_based import WhiteboxSeeker, BlackboxSeeker
from utils import UnfairMetric, load_model, get_L_matrix
from data import adult
from train_dnn import get_data
from dnn_models.model import MLP, NormMLP
from distances.normalized_mahalanobis_distances import SquaredEuclideanDistance, ProtectedSEDistances
from distances.sensitive_subspace_distances import LogisticRegSensitiveSubspace
from distances.binary_distances import BinaryDistance
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from IPython.display import display

%load_ext autoreload
%autoreload 2

In [2]:
rand_seed = 0
use_protected_attr = True
protected_vars = ['race_White']
# protected_vars = ['sex_Male', 'race_White']

dataset, train_dl, test_dl = get_data(adult, rand_seed, protected_vars=protected_vars)
dataset.use_protected_attr = use_protected_attr
in_dim = dataset.dim_feature()
out_dim = 2

# prepare data
all_X, all_y = dataset.get_all_data(), dataset.labels

adult_gen = adult.Generator(sensitive_columns=dataset.protected_idxs, include_protected_feature=use_protected_attr)

In [3]:
model = MLP(in_dim, out_dim)
load_model(model, 'MLP', 'adult', 'STDTrainer', use_protected_attr=use_protected_attr, \
           protected_vars=protected_vars, id=rand_seed)

# model = NormMLP(in_dim, out_dim, adult_gen.get_range('data'))
# load_model(model, 'NormMLP', 'adult', 'STDTrainer', use_protected_attr=use_protected_attr, \
#            protected_vars=protected_vars, id=rand_seed)

# all_pred = model.get_prediction(all_X)

In [4]:
# prepare distances
distance_x_NSE = SquaredEuclideanDistance()
distance_x_Causal = ProtectedSEDistances()
distance_x_LR = LogisticRegSensitiveSubspace()
distance_y = BinaryDistance()

distance_x_NSE.fit(num_dims=dataset.dim_feature(), data_gen=adult_gen)
distance_x_Causal.fit(num_dims=dataset.dim_feature(), data_gen=adult_gen, protected_idx=dataset.protected_idxs)
distance_x_LR.fit(all_X, adult_gen, protected_idxs=dataset.protected_idxs)

In [5]:
def rand_gen():
    return {
        'age': random.randint(15, 60),
        'capital_gain': 0,
        'capital_loss': 0,
        'education_num': random.randint(1, 15),
        'hours_per_week': random.randint(10, 50),
        'race_white': random.choice([0, 1]),
        'sex_male': random.choice([0, 1]),
        'marital_status': random.choice(list(range(7))),
        'occupation': random.choice(list(range(14))),
        'relationship': random.choice(list(range(6))),
        'workclass': random.choice(list(range(7)))
    }

def perturb_pair(x, pert_features, pert_func):
    pair = dict()
    for k, v in x.items():
        pair[k] = torch.tensor([v, v])
        if k in pert_features:
            pair[k][1] = pert_func(pair[k][0])
    return pair

x = rand_gen()
# pair = adult.generate_from_origin(**perturb_pair(x, ['sex_male', 'race_white'], lambda x:1-x))
# adult.get_original_feature(pair)

In [6]:
pair = adult.generate_from_origin(**perturb_pair(x, ['capital_loss'], lambda x:1+x))
adult.get_original_feature(pair)

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,38.0,0.0,0.0,12.0,27.0,0.0,1.0,4.0,11.0,0.0,1.0
1,38.0,0.0,1.0,12.0,27.0,0.0,1.0,4.0,11.0,0.0,1.0


In [7]:
# dist = distance_x_LR(all_X[0], all_X[0] + torch.eye(all_X.shape[1]), itemwise_dist=False).squeeze()
# print(dist)
# print(dist[dataset.protected_idxs])

In [8]:
# 1/1e-10

In [9]:
dist = distance_x_LR(all_X[0], all_X[0] + torch.eye(all_X.shape[1]), itemwise_dist=False).squeeze()
print(dist)
print(dist[dataset.protected_idxs])

tensor([1.8081e-04, 9.9866e-11, 5.1967e-08, 4.4074e-03, 1.0411e-04, 9.1793e-01,
        9.9521e-01, 1.0000e+00, 9.9131e-01, 9.9849e-01, 9.9440e-01, 9.2576e-01,
        9.9944e-01, 9.7467e-01, 9.8442e-01, 9.7501e-01, 9.4209e-01, 9.9902e-01,
        9.8413e-01, 9.8653e-01, 9.9295e-01, 9.8633e-01, 9.9348e-01, 9.8243e-01,
        9.9697e-01, 9.9736e-01, 0.0000e+00, 9.3236e-01, 9.9929e-01, 9.2443e-01,
        1.0000e+00, 8.9661e-01, 9.9140e-01, 9.9605e-01, 9.5576e-01, 1.0000e+00,
        9.7955e-01, 8.9331e-01, 8.9278e-01, 9.9982e-01, 9.8102e-01])
tensor([0.])


设置之后允许每一维变多少

In [10]:
# x = rand_gen()
# for key in x:
#     pair = adult.generate_from_origin(**perturb_pair(x, key, x[key]+1))
#     print(distance_x_NSE(pair[0], pair[1]), 1 / distance_x_NSE(pair[0], pair[1]))

In [11]:
# epsilon越大，要求不公平样本对dx越小，越严格
# epsilon = 1e10
epsilon = 9e9
unfair_metric = UnfairMetric(dx=distance_x_LR, dy=distance_y, epsilon=epsilon)

In [12]:
# L = get_L_matrix(all_X, all_pred, distance_x_Causal, distance_y)
# n_unfair = torch.sum(L>epsilon).item()

# total_pairs = all_X.shape[0]**2

# unfair_ratio = n_unfair/total_pairs
# unfair_ratio

In [13]:
# 1/unfair_ratio

In [14]:
# p_success = lambda n: 1 - (1 - unfair_ratio)**n
# for i in range(10):
#     if p_success(10**i) > 1/100:
#         print(10**i, p_success(10**i))

In [15]:
def show_result(result):
    pair, n_query = result
    if pair != None:
        display(adult.get_original_feature(pair), n_query)
    else:
        print('not found')

In [16]:
# random.seed(422)
# torch.manual_seed(422)

# select_seeker = Random_select_seeker(model=model, unfair_metric=unfair_metric, data=all_X)
# for _ in range(5):
#     show_result(select_seeker.seek(dx_constraint=True, max_query=1e6))

In [17]:
# random.seed(422)
# torch.manual_seed(422)

# gen_seeker = Random_gen_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# for _ in range(3):
#     show_result(gen_seeker.seek(by_range=True, max_query=1e6))

In [18]:
# random.seed(422)
# torch.manual_seed(422)

# for _ in range(3):
#     show_result(gen_seeker.seek(by_range=False, max_query=1e6))

In [19]:
# adult_gen.get_range('data')

In [20]:
# random.seed(422)
# torch.manual_seed(422)

# white_seeker = White_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# # show_result(white_seeker.seek())
# for _ in range(5):
#     show_result(white_seeker.seek(origin_lr=10, max_query=1e6))

In [21]:
# random.seed(422)
# torch.manual_seed(422)

# black_seeker = Black_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# for _ in range(5):
#     show_result(black_seeker.seek(max_query=1e6))

In [22]:
# random.seed(422)
# torch.manual_seed(422)

# test_seeker = Norm_Test_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# show_result(white_seeker.seek())
# for _ in range(1):
#     show_result(test_seeker.seek(black_box=False, origin_lr1=1, max_query=1e6))

In [26]:
random.seed(422)
torch.manual_seed(422)

test_seeker = Norm_Test_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
for _ in range(1):
    show_result(test_seeker.seek(black_box=True, origin_lr1=1, max_query=1e6))

continue 84
continue 167
continue 250
continue 333
continue 416
continue 499
continue 582
lr/=5 665
continue 666
continue 749
continue 832
continue 915
lr/=5 998
continue 999
continue 1082
lr/=5 1165
continue 1166
lr/=5 1249
continue 1250
continue 1333
continue 1416
lr/=5 1499
continue 1500
lr/=5 1583
continue 1584
continue 1667
lr/=5 1750
lr/=5 1751
converge 1752
tensor([   38, 37125,  2409,     6,     5,     0,     0,     0,     0,     1,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     1,     0,     0,     1,     0,     0,
            0,     0,     0,     0,     0,     0,     1,     0,     0,     0,
            0], dtype=torch.int32) tensor(0, dtype=torch.int32)
tensor([[-7.2719, 12.6854]], grad_fn=<AddmmBackward0>)
tensor([  38,   64, 2157,    6,    6,    0,    0,    0,    0,    1,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           1,    0,    0,    1,    0,    0

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,38.0,64.0,2157.0,6.0,6.0,0.0,0.0,4.0,12.0,0.0,2.0
1,38.0,64.0,2157.0,6.0,6.0,1.0,0.0,4.0,12.0,0.0,2.0


1834

In [27]:
# random.seed(422)
# torch.manual_seed(422)

# test_seeker = WhiteboxSeeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# # show_result(white_seeker.seek())
# for _ in range(1):
#     show_result(test_seeker.seek(origin_lr=1, max_query=1000, lamb=1))

In [31]:
random.seed(422)
torch.manual_seed(422)

test_seeker = BlackboxSeeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# show_result(white_seeker.seek())
for _ in range(1):
    show_result(test_seeker.seek(origin_lr=2, max_query=2000, lamb=1))

continue 84
continue 167
continue 250
lr/=5 333
continue 334
continue 417
lr/=5 500
continue 501
lr/=5 584
continue 585
continue 668
continue 751
continue 834
continue 917
lr/=5 1000
lr/=5 1001
continue 1002
lr/=5 1085
continue 1086
continue 1169
continue 1252
continue 1335
lr/=5 1418
continue 1419
continue 1502
lr/=5 1585
converge: 1586
tensor([   38, 37125,  2409,     6,     5,     0,     0,     0,     0,     1,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     1,     0,     0,     1,     0,     0,
            0,     0,     0,     0,     0,     0,     1,     0,     0,     0,
            0], dtype=torch.int32) tensor(0, dtype=torch.int32)
tensor([[-7.2719, 12.6854]], grad_fn=<AddmmBackward0>)
tensor([  38,   59, 2220,    6,    6,    0,    0,    0,    0,    1,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           1,    0,    0,    1,    0,    0,    0,    0,    0,    0,  

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,38.0,59.0,2220.0,6.0,6.0,0.0,0.0,4.0,12.0,0.0,2.0
1,38.0,59.0,2220.0,6.0,6.0,1.0,0.0,4.0,12.0,0.0,2.0


1668